In [109]:
%reset

In [110]:
import os
import pandas as pd
import numpy as np
import scipy.stats as stats
import math
import time
from sklearn.mixture import GaussianMixture
import warnings
warnings.filterwarnings('ignore')

In [111]:
class Histogram(object):
        
    def __init__(self, min_of_data, max_of_data, ensembleMembers, numberOfBins):
        self.num_of_bins = numberOfBins
        self.ensembleMembers = ensembleMembers
        self.max_of_data = max_of_data
        self.min_of_data = min_of_data
        self.range_of_data = self.max_of_data - self.min_of_data
        
    
    def get_time_step(self, grid_1D_array):
        time_step_1D = np.empty(len(grid_1D_array))
        count = 0
        time = 0
        for i in range(len(grid_1D_array) // self.ensembleMembers):
            time_step_1D[count:count+self.ensembleMembers] = time
            count += self.ensembleMembers
            time += 1
        return time_step_1D
    
    
    
    def compute_bin_index(self, grid_1D_array):
        bin_index = np.array([])
        interval = self.range_of_data / self.num_of_bins
        bin_index = (grid_1D_array - self.min_of_data) // interval
        bin_index[bin_index > self.num_of_bins - 1] = self.num_of_bins - 1
        bin_index[bin_index < 0 ] = 0
        return bin_index
    
    
    def get_bin_timestep(self, data_time_step, bin_1D_index):
        data = pd.Series(data_time_step, index = bin_1D_index)
        sorted_data = data.sort_index()
        unique_bin_index = np.unique(sorted_data.index)
        return sorted_data, unique_bin_index
    

In [112]:
class Cluster(object):
    cluster_index = 0
    
    def __init__(self, bin, x, y, bin_position, interval, z=0):
        self.update(bin, x, y, bin_position, interval)
        self.x = x
        self.y = y
        self.z = z
        self.bin_position = bin_position
        self.S_interval = interval
        self.time = bin
        self.no = self.cluster_index
        self.label = self.cluster_index
        Cluster.cluster_index += 1
    
    def update(self, bin, x, y, bin_position, interval):
        self.x = x
        self.y = y
        self.bin_position = bin_position
        self.S_interval = interval
        self.time = bin

class SLICProcessor(object):
    
    def __init__(self, data, K, SBN, timestep, num_of_bins):
        self.data = data
        self.K = K
        self.select_binNum = SBN
        self.timestep = timestep
        self.data_size = data.shape
        if len(self.data_size) > 4:
            self.data_x = self.data_size[0]
            self.data_y = self.data_size[1]
            self.data_z = self.data_size[2]
            self.N = self.data_x * self.data_y * self.data_z
            self.S = int(math.sqrt(self.N / self.K))
            self.label = np.full((self.data_x, self.data_y, self.data_z, num_of_bins), -1)
            self.dis = np.full((self.data_x, self.data_y, self.data_z, num_of_bins), np.inf)
        else:
            self.data_x = self.data_size[0]
            self.data_y = self.data_size[1]
            self.N = self.data_x * self.data_y
            self.S = int(math.sqrt(self.N / self.K))
            self.label = np.full((self.data_x, self.data_y, num_of_bins), -1)
            self.dis = np.full((self.data_x, self.data_y, num_of_bins), np.inf)
        self.clusters = []
        self.clustersCenter = []
    
    def make_cluster_2D(self, bin_object, x, y, bin_position, interval):
        return Cluster(bin_object, x, y, bin_position, interval)
    
    def make_cluster_3D(self, bin_objext, x, y, z , bin_position, interval):
        return Cluster(bin_objext, x, y, bin_position, interval, z)
    
    def init_clusters(self):
        if len(self.data_size) > 4:
            Cluster.cluster_index = 0
            sx = int(self.S / 2)
            sy = int(self.S / 2)
            sz = int(self.S / 2)
            while sx < self.data_x:
                while sy < self.data_y:
                    while sz < self.data_z:
                        grid = self.data[sx,sy,sz]
                        interval_of_center = math.ceil((len(grid[grid != -1])/self.timestep) / self.select_binNum)
                        if(interval_of_center < 1):
                            interval_of_center = 1
                        sb = math.ceil(interval_of_center / 2)
                        while sb < len(grid): 
                            center_bin = grid[sb]
                            if(center_bin[0] == -1):
                                sb = sb + interval_of_center
                            else:
                                self.clusters.append(self.make_cluster_3D(center_bin, sx, sy, sz , sb, interval_of_center))
                                # print('cluster info: ', 'x: ', sx, 'y: ', sy, 'z: ', sz, 'sb: ', sb)
                            sb = sb + interval_of_center
                        sz = sz + self.S
                    sz = int(self.S / 2)
                    sy = sy + self.S
                sy = int(self.S / 2)
                sx = sx + self.S
            print(len(self.clusters))
        else:
            Cluster.cluster_index = 0
            sx = int(self.S / 2)
            sy = int(self.S / 2)
            label = 0   
            while sx < self.data_x:
                while sy < self.data_y:
                    # print('sx: ', sx, 'sy: ', sy)
                    grid = self.data[sx,sy]
                    interval_of_center = math.ceil((len(grid[grid != -1])/self.timestep) / self.select_binNum)
                    if(interval_of_center < 1):
                        interval_of_center = 1
                    sb = int(interval_of_center / 2)
                    #make cluster center
                    while sb < len(grid):
                        center_bin = grid[sb]
                        if(center_bin[0] == -1):
                            sb = sb+interval_of_center
                        else:
                            self.clusters.append(self.make_cluster_2D(center_bin, sx, sy, sb, interval_of_center))
                            # print('cluster info: ', 'x: ', sx, 'y: ', sy, 'sb: ', sb)
                        sb = sb + interval_of_center
                    sy = sy + self.S   
                sy = int(self.S / 2)
                sx = sx + self.S
    
   
    def assignment(self, threshold):
        if len(self.data_size) > 4:
            for cluster in self.clusters:
                for x in range(cluster.x - 1 * self.S, cluster.x + 1 * self.S):
                    if x < 0 or x >= self.data_x: continue
                    for y in range(cluster.y - 1 * self.S, cluster.y + 1 * self.S):
                        if y < 0 or y >= self.data_y: continue
                        for z in range(cluster.z - 1 * self.S, cluster.z + 1 * self.S):
                            if z < 0 or z >= self.data_z: continue
                            grid = self.data[x,y,z]
                            for binPos in range(cluster.bin_position - 1*cluster.S_interval, cluster.bin_position + 1*cluster.S_interval):
                                if binPos < 0 or binPos >= (len(grid) - 1): continue
                                dists = [i for i in range(self.timestep)]
                                if (grid[binPos][0]==-1): continue
                                D = stats.wasserstein_distance(dists, dists, grid[binPos], cluster.time)
                                if (D < threshold and D < self.dis[x, y, z, binPos]):
                                    self.label[x,y,z,binPos] = cluster.no
                                    self.dis[x,y,z,binPos] = D
        else:
            for cluster in self.clusters:
                for x in range(cluster.x - 1 * self.S, cluster.x + 1 * self.S):
                    if x < 0 or x >= self.data_x: continue
                    for y in range(cluster.y - 1 * self.S, cluster.y + 1 * self.S):
                        if y < 0 or y >= self.data_y: continue
                        grid = self.data[x,y]
                        for binPos in range(cluster.bin_position - 1*cluster.S_interval, cluster.bin_position + 1*cluster.S_interval): #for binPos in range(0, len(grid.histogram_bins)):
                            if binPos < 0 or binPos >= (len(grid) - 1): continue
                            dists = [i for i in range(self.timestep)]
                            if (grid[binPos][0]==-1): continue
                            D = stats.wasserstein_distance(dists, dists, grid[binPos], cluster.time)
                            if (D < threshold and D < self.dis[x, y, binPos]):
                                self.label[x,y,binPos] = cluster.no
                                self.dis[x, y, binPos] = D

In [113]:
class TGMM(object):
    def __init__(self, file, ensembleMembers, numberOfBins, timestep, maxOfData, minOfData):
        self.num_of_bins = numberOfBins
        self.ensembleMembers = ensembleMembers
        self.data = file
        self.max_of_data = maxOfData
        self.min_of_data = minOfData
        self.timestep = timestep
        self.k = 0
        self.select_bin_num = 0
        self.comp = 0
        self.total_bins = 0
        if len(self.data.shape) > 3:
            self.x = self.data.shape[0]
            self.y = self.data.shape[1]
            self.z = self.data.shape[2]
            self.data_size = [self.x, self.y, self.z]
            self.time_distribution = np.full((self.x, self.y, self.z, self.num_of_bins, timestep), -1)
            self.prob = np.full((self.x, self.y, self.z, self.num_of_bins), 0.0)
            self.label = np.full((self.x, self.y, self.z, self.num_of_bins), -1)
            self.output_label = np.full((self.x, self.y, self.z, self.num_of_bins), -1)
            self.reconstructed_data = np.full((self.x, self.y, self.z, self.num_of_bins), np.inf)
            self.Tgmm_bindata = np.full((self.x, self.y, self.z, self.num_of_bins, 4), -1.00) #4 = index, prob, Gmm_label, Num_of_component
        else:
            self.x = self.data.shape[0]
            self.y = self.data.shape[1]
            self.data_size = [self.x, self.y]
            self.time_distribution = np.full((self.x, self.y, self.num_of_bins, self.timestep), -1)
            self.prob = np.full((self.x, self.y, self.num_of_bins), 0.0)
            self.label = np.full((self.x, self.y, self.num_of_bins), -1)
            self.output_label = np.full((self.x, self.y, self.num_of_bins), -1)
            self.reconstructed_data = np.full((self.x, self.y, self.num_of_bins), np.inf)
            self.Tgmm_bindata = np.full((self.x, self.y, self.num_of_bins, 4), -1.00)
        self.grids = []
        self.slic_label = []
        self.share_Gmm = []
        self.non_share_Gmm = []
        self.GMM = []
        self.center = []    
        
    def get_hist_info(self):
        hist = Histogram(self.min_of_data, self.max_of_data, self.ensembleMembers, self.num_of_bins)
        if len(self.data.shape) > 3: 
            data_time_step_info = hist.get_time_step(self.data[0,0,0])
            for x in range(self.x):
                for y in range(self.y):
                    for z in range(self.z):
                        data_time_step = data_time_step_info
                        bin_1D_index = hist.compute_bin_index(self.data[x,y,z])
                        time_step_of_bins, uni_index = hist.get_bin_timestep(data_time_step, bin_1D_index)
                        self.total_bins = self.total_bins+len(uni_index)
                        for index in uni_index:
                            i = int(index)
                            time_data = []
                            time_data = np.append(time_data, time_step_of_bins[i])
                            time_data = time_data.astype('int')
                            temp_dist = np.bincount(time_data)
                            prob = (sum(temp_dist) / len(self.data[x,y,z]))
                            time_dist = np.pad(temp_dist, (0, int((data_time_step.max()+1) - len(temp_dist))))
                            self.time_distribution[x,y,z,i] = time_dist
                            self.prob[x,y,z,i] = prob
        else:
            data_time_step_info = hist.get_time_step(self.data[0,0])
            for x in range(self.x):
                for y in range(self.y):
                    data_time_step = data_time_step_info
                    bin_1D_index = hist.compute_bin_index(self.data[x,y])
                    time_step_of_bins, uni_index = hist.get_bin_timestep(data_time_step, bin_1D_index)
                    self.total_bins = self.total_bins+len(uni_index)
                    for index in uni_index:
                        i = int(index)
                        time_data = []
                        time_data = np.append(time_data, time_step_of_bins[i])
                        time_data = time_data.astype('int')
                        temp_dist = np.bincount(time_data)
                        prob = (sum(temp_dist) / len(self.data[x,y]))
                        time_dist = np.pad(temp_dist, (0, int((data_time_step.max()+1) - len(temp_dist))))
                        self.time_distribution[x,y,i] = time_dist
                        self.prob[x,y,i] = prob  


    def SLIC(self, k, select_bin_num, threshold):
        self.k = k
        self.select_bin_num = select_bin_num
        slic = SLICProcessor(self.time_distribution, k, select_bin_num, self.timestep, self.num_of_bins)
        slic.init_clusters()
        slic.assignment(threshold)
        self.center = slic.clusters
        self.label = slic.label
        
    def set_training_data(self, time_step):
        training_data = []
        for i in range(len(time_step)):
            if time_step[i] == 0: continue
            for j in range(time_step[i]):
                training_data = np.append(training_data, i)
                if(len(training_data) < 2):
                    training_data = np.append(training_data, training_data)
        training_data = np.array(training_data).reshape(-1,1)
        return training_data    
    
    
    ### fitting the GMM
    def fit_center_model(self, comp):
        self.comp = comp
        self.share_Gmm = []
        for center_cluster in self.center:
            time_data = center_cluster.time
            self.share_Gmm.append(self.GMM_fit(time_data))
        
    
    def fit_nonCluster_bin(self):
        index = len(self.share_Gmm)
        self.non_share_Gmm = []
        self.output_label = np.array(self.label)
        if len(self.data.shape) > 3:
            for x in range(self.x):
                for y in range(self.y):
                    for z in range(self.z):
                        for bin_i in range(self.num_of_bins):
                            model_label = self.label[x,y,z,bin_i]
                            if ( model_label == -1):
                                if (self.time_distribution[x,y,z,bin_i,0] == -1):continue
                                self.output_label[x,y,z,bin_i] = index
                                time_data = self.time_distribution[x,y,z,bin_i]
                                self.non_share_Gmm.append(self.GMM_fit(time_data))
                                index = index+1   
            print(len(self.non_share_Gmm))         
        else:
            for x in range(self.x):
                for y in range(self.y):
                    for bin_i in range(self.num_of_bins):
                        model_label = self.label[x,y,bin_i]
                        if ( model_label == -1):
                            if (self.time_distribution[x,y,bin_i,0] == -1):continue
                            self.output_label[x,y,bin_i] = index
                            time_data = self.time_distribution[x,y,bin_i]
                            self.non_share_Gmm.append(self.GMM_fit(time_data))
                            index = index+1
    
    def fit_model_nonslic(self,comp):
        self.comp = comp
        index = 0
        self.GMM = []
        if len(self.data.shape) > 3:
            for x in range(self.x):
                for y in range(self.y):
                    for z in range(self.z):
                        for bin_i in range(self.num_of_bins):
                            if (self.time_distribution[x,y,z,bin_i,0] == -1):continue
                            self.output_label[x,y,z,bin_i] = index
                            time_data = self.time_distribution[x,y,z,bin_i]
                            self.GMM.append(self.GMM_fit(time_data))
                            index = index+1 
        else:    
            for x in range(self.x):
                for y in range(self.y):
                    for bin_i in range(self.num_of_bins):
                        if (self.time_distribution[x,y,bin_i,0] == -1):continue
                        self.output_label[x,y,bin_i] = index
                        time_data = self.time_distribution[x,y,bin_i]
                        self.GMM.append(self.GMM_fit(time_data))
                        index = index+1

    def combine_Gmm(self):
        self.GMM = []
        self.GMM = np.append(self.share_Gmm, self.non_share_Gmm)
            
    def GMM_fit(self,time_data):
        training_data = self.set_training_data(time_data)
        if(len(training_data) < self.comp):
            model = GaussianMixture(len(training_data), covariance_type='full', random_state=0, reg_covar=1).fit(training_data)
        else:
            model = GaussianMixture(self.comp, covariance_type='full', random_state=0, reg_covar=1).fit(training_data)
        return model 
    
    def fit_GMM(self, Using_slic, num_of_components):
        if (Using_slic == True):
            self.fit_center_model(num_of_components)
            self.fit_nonCluster_bin()
            self.combine_Gmm()
        else:
            self.fit_model_nonslic(num_of_components)
    
    
    ### compute storage
    def compute_storage(self, path, timesteps_range):
        storage_txt = path+'/'+'storage_'+timesteps_range+'.txt'
        text_file = open(storage_txt,'w')
        total_GMM = len(self.GMM)
        share_GMM = len(self.share_Gmm)
        non_share_GMM = len(self.non_share_Gmm)
        storage_of_GMM = total_GMM * (self.comp*3)
        storage_of_bins = self.total_bins * 2
        total_storage = storage_of_bins + storage_of_GMM
        storage_Byte = ((total_storage * 32) / 8)/1000000
        print('Number of total GMM: ', total_GMM, file=text_file)
        print('Number of share GMM: ', share_GMM, file=text_file)
        print('Numbere of nonshare GMM: ', non_share_GMM, file=text_file)
        print('Total number of bins: ', self.total_bins, file=text_file)
        print('Storage: ', storage_Byte, 'MB', file=text_file)
        text_file.close()
        
     
    ### Save data
    def save_Tgmm_data(self, path, timesteps_range):
        bin_info = []
        temp_bin_info = []
        if len(self.data.shape) > 3:
            for x in range(self.x):
                for y in range(self.y):
                    for z in range(self.z):
                        grid_index = x*self.y*self.z + y*self.z + z 
                        for bin_index in range(self.num_of_bins):
                            if (self.time_distribution[x,y,z,bin_index,0] == -1):continue
                            prob = self.prob[x,y,z,bin_index]
                            label = self.output_label[x,y,z,bin_index]
                            temp_bin_info.append([grid_index, bin_index, prob, label, self.GMM[label].n_components])
            name = 'bin_data_x{x}_y{y}_z{z}_NumOfBins{nob}_RangeOfTimes{timesteps_range}.bin'.format(x = self.x, y = self.y, z = self.z, nob = self.num_of_bins, timesteps_range = timesteps_range)
        else:
            for x in range(self.x):
                for y in range(self.y):
                    grid_index = x*self.y + y
                    for bin_index in range(self.num_of_bins):
                        if (self.time_distribution[x,y,bin_index,0] == -1):continue
                        prob = self.prob[x,y,bin_index]
                        label = self.output_label[x,y,bin_index]
                        temp_bin_info.append([grid_index, bin_index, prob, label, self.GMM[label].n_components])
            bin_info = np.array(temp_bin_info)
            name = 'bin_data_x{x}_y{y}_NumOfbBins{nob}_{timesteps_range}.bin'.format(x = self.x, y = self.y, nob = self.num_of_bins, timesteps_range = timesteps_range)
        bin_info.astype('float32').tofile(path+'/'+name)

    def save_GMM(self, path, timesteps_range):
        GMM = []
        weights = []
        means = []
        covars = []
        label = []
        index = 0
        for gmm in self.GMM:
            label.append(index)
            index = index + gmm.n_components 
            for n_comps in range(gmm.n_components):
                W = np.ravel(gmm.weights_[n_comps])[0]
                M = np.ravel(gmm.means_[n_comps])[0]
                C = np.ravel(gmm.covariances_[n_comps])[0]
                weights.append(W)
                means.append(M)
                covars.append(C)
        GMM = np.vstack((weights, means, covars))
        name = 'Gmm_' + timesteps_range + '.bin'
        labelName = 'label_' + timesteps_range + '.bin'
        GMM.astype('float32').tofile(path+'/'+name)
        label = np.array(label)
        label.astype('float32').tofile(path+'/'+labelName)

    def binedge(self):
        hedge = np.zeros(self.num_of_bins+1)
        for i in range(len(hedge)):
            if(i == 0):
                hedge[i] = self.min_of_data
            else:
                hedge[i] = hedge[i-1] + ((self.max_of_data-self.min_of_data)/self.num_of_bins)
        return hedge
    
    def save_edge(self, path, timesteps_range):
        edge = self.binedge()
        name = 'edge_' + timesteps_range + '.bin'
        edge.astype(np.float32).tofile(path+'/'+name)
    
    def save_data(self, all_path, time_range):
        timesteps_range = 'time_range_' + time_range
        print('Save path: ', all_path)
        self.save_Tgmm_data(all_path, timesteps_range)
        self.save_GMM(all_path, timesteps_range)
        self.save_edge(all_path, timesteps_range)
        self.compute_storage(all_path, timesteps_range)

In [114]:
def fit_TGMM(data, Num_of_ensembles, Num_of_bins, Num_of_timesteps, consecutive_timesteps, Num_of_GMMcomps, Save_path, floder_name, SLIC_parameter=[0,0,0]):
    time_start = time.time()
    all_path = os.path.join(Save_path, floder_name)
    os.mkdir(all_path)
    for start_timestep in range(0, Num_of_timesteps, consecutive_timesteps):
        range_of_times = '{start_timestep}_{end_timestep}'.format(start_timestep = start_timestep, end_timestep = start_timestep+consecutive_timesteps)
        print('timesteps range: ', range_of_times)
        if (len(data.shape) > 3): 
            training_data = data[:,:,:,start_timestep*Num_of_ensembles:(start_timestep+consecutive_timesteps)*Num_of_ensembles]
            max_of_data = training_data.max()
            min_of_data = training_data.min()
            SLIC_TGMM = TGMM(training_data, Num_of_ensembles, Num_of_bins, Num_of_timesteps, max_of_data, min_of_data)
        else:
            training_data = data[:,:,start_timestep*Num_of_ensembles:(start_timestep+consecutive_timesteps)*Num_of_ensembles]
            max_of_data = training_data.max()
            min_of_data = training_data.min()
            SLIC_TGMM = TGMM(training_data, Num_of_ensembles, Num_of_bins, consecutive_timesteps, max_of_data, min_of_data)
        print('Compute Histogram Infomation')
        SLIC_TGMM.get_hist_info()
        hist_time = (time.time() - time_start) / 60
        if(SLIC_parameter!=[0,0,0]):
            print('SLIC Classification')
            k = SLIC_parameter[0]
            selected_bins = SLIC_parameter[1]
            threshold = SLIC_parameter[2]
            SLIC_TGMM.SLIC(k, selected_bins, threshold)
            SLIC_time = (time.time() - time_start) / 60
            print('Fitting Gaussian Mixture Model')
            SLIC_TGMM.fit_GMM(True, Num_of_GMMcomps)
            fitting_time = (time.time() - time_start) /60
        else:
            print('Fitting Gaussian Mixture Model')
            SLIC_TGMM.fit_GMM(False, Num_of_GMMcomps)
            fitting_time = (time.time() - time_start)
        time_end = time.time()
        time_txt = all_path+'/time_'+range_of_times+'.txt'
        text_file = open(time_txt, 'w')
        print('Compute Histogram times: ', hist_time, 'minutes', file=text_file)
        print('SLIC times: ', SLIC_time, 'minutes', file=text_file)
        print('Fitting GMM times: ', fitting_time, 'minutes', file=text_file)
        print('execution time: ', (time_end - time_start)/60, 'minutes', file=text_file)
        text_file.close()
        SLIC_TGMM.save_data(all_path, range_of_times)

In [115]:
air_data = np.load('./Data_set/Air_temp_160_320_ensemble60_timestep150.npy')

In [116]:
path = '/home/60947081s/Air_temp_Experiment/'
Num_of_ensembles = 60
Num_of_bins = 128
Num_of_timesteps = 150
consecutive_time = 75
Slic_paremeter = [1024,100,3]
for i in [1,3,5,7]:
    floder_name = 'Air_ta_x160_y320_NumOfEns{NumOfEns}_Timesteps{timesteps}_ConsTimes{consTime}_NumOfComps{comps}_SLIC_{k}_{NOB}_{threshold}'.format(NumOfEns=Num_of_ensembles,timesteps=Num_of_timesteps, consTime=consecutive_time , comps = i, k=Slic_paremeter[0], NOB=Slic_paremeter[1], threshold=Slic_paremeter[2])
    print(floder_name)
    Num_of_GMMcomps = i
    fit_TGMM(air_data, Num_of_ensembles, Num_of_bins, Num_of_timesteps, consecutive_time, 1, path, floder_name, Slic_paremeter)

timesteps range:  0_150
Compute Histogram Infomation
SLIC Classification
Fitting Gaussian Mixture Model
Save path:  ./test/Air_ta_x160_y320_NumOfEns60_Timesteps150_ConsTimes150_NumOfComps1_SLIC_1024_100_3


In [ ]:
%%time 

start = time.time()
path = '/home/60947081s/Air_temp_Experiment/'
floder_name = 'Air_ta_x160_y320_NumOfEns60_Timesteps150_NumOfComps1_SLIC_4096_100_5'
Slic_comp1 = TGMM(air_data, 60, 128, 150, max, min)
print('start get hist info')
Slic_comp1.get_hist_info()
hist_time = (time.time()-start)/60
print('SLIC')
Slic_comp1.SLIC(4096, 100, 3)
Slic_time = (time.time()-start)/60
print('fitting model')
Slic_comp1.fit_GMM(True, 1)
fitting_time = (time.time()-start)/60
Slic_comp1.save_data(path, floder_name)
time_txt = path+floder_name+'/'+'time.txt'

end = time.time()
text_file = open(time_txt,'w')
print('Hist times: ', hist_time, 'minutes', file=text_file)
print('SLIC times: ', Slic_time, 'minutes', file=text_file)
print('Fitting times: ', fitting_time, 'minutes', file=text_file)
print('All times: ', (end - start)/60, 'minutes', file=text_file)
text_file.close()

In [ ]:
%%time 

start = time.time()
path = '/home/60947081s/Air_temp_Experiment/'
floder_name = 'Air_ta_x160_y320_NumOfEns60_Timesteps150_NumOfComps3_SLIC_4096_100_5'
Slic_comp3 = TGMM(air_data, 60, 128, 150, max, min)
print('start get hist info')
Slic_comp3.get_hist_info()
hist_time = (time.time()-start)/60
print('SLIC')
Slic_comp3.SLIC(4096, 100, 5)
Slic_time = (time.time()-start)/60
print('fitting model')
Slic_comp3.fit_GMM(True, 3)
fitting_time = (time.time()-start)/60
Slic_comp3.save_data(path, floder_name)
time_txt = path+floder_name+'/'+'time.txt'

end = time.time()
text_file = open(time_txt,'w')
print('Hist times: ', hist_time, 'minutes', file=text_file)
print('SLIC times: ', Slic_time, 'minutes', file=text_file)
print('Fitting times: ', fitting_time, 'minutes', file=text_file)
print('All times: ', (end - start)/60, 'minutes', file=text_file)
text_file.close()

In [ ]:
%%time 

start = time.time()
path = '/home/60947081s/Air_temp_Experiment/'
floder_name = 'Air_ta_x160_y320_NumOfEns60_Timesteps150_NumOfComps3_SLIC_4096_100_5'
Slic_comp5 = TGMM(air_data, 60, 128, 150, max, min)
print('start get hist info')
Slic_comp5.get_hist_info()
hist_time = (time.time()-start)/60
print('SLIC')
Slic_comp5.SLIC(4096, 100, 5)
Slic_time = (time.time()-start)/60
print('fitting model')
Slic_comp5.fit_GMM(True, 5)
fitting_time = (time.time()-start)/60
Slic_comp5.save_data(path, floder_name)
time_txt = path+floder_name+'/'+'time.txt'

end = time.time()
text_file = open(time_txt,'w')
print('Hist times: ', hist_time, 'minutes', file=text_file)
print('SLIC times: ', Slic_time, 'minutes', file=text_file)
print('Fitting times: ', fitting_time, 'minutes', file=text_file)
print('All times: ', (end - start)/60, 'minutes', file=text_file)
text_file.close()

In [ ]:
%%time 

start = time.time()
path = '/home/60947081s/Air_temp_Experiment/'
floder_name = 'Air_ta_x160_y320_NumOfEns60_Timesteps150_NumOfComps3_SLIC_4096_100_5'
Slic_comp7 = TGMM(air_data, 60, 128, 150, max, min)
print('start get hist info')
Slic_comp7.get_hist_info()
hist_time = (time.time()-start)/60
print('SLIC')
Slic_comp7.SLIC(4096, 100, 5)
Slic_time = (time.time()-start)/60
print('fitting model')
Slic_comp7.fit_GMM(True, 7)
fitting_time = (time.time()-start)/60
Slic_comp7.save_data(path, floder_name)
time_txt = path+floder_name+'/'+'time.txt'

end = time.time()
text_file = open(time_txt,'w')
print('Hist times: ', hist_time, 'minutes', file=text_file)
print('SLIC times: ', Slic_time, 'minutes', file=text_file)
print('Fitting times: ', fitting_time, 'minutes', file=text_file)
print('All times: ', (end - start)/60, 'minutes', file=text_file)
text_file.close()

In [ ]:
%%time 

start = time.time()
path = '/home/60947081s/Air_temp_Experiment/'
floder_name = 'Air_ta_x160_y320_NumOfEns60_Timesteps150_NumOfComps1_SLIC_4096_100_7'
Slic_comp1 = TGMM(air_data, 60, 128, 150, max, min)
print('start get hist info')
Slic_comp1.get_hist_info()
hist_time = (time.time()-start)/60
print('SLIC')
Slic_comp1.SLIC(4096, 100, 7)
Slic_time = (time.time()-start)/60
print('fitting model')
Slic_comp1.fit_GMM(True, 1)
fitting_time = (time.time()-start)/60
Slic_comp1.save_data(path, floder_name)
time_txt = path+floder_name+'/'+'time.txt'

end = time.time()
text_file = open(time_txt,'w')
print('Hist times: ', hist_time, 'minutes', file=text_file)
print('SLIC times: ', Slic_time, 'minutes', file=text_file)
print('Fitting times: ', fitting_time, 'minutes', file=text_file)
print('All times: ', (end - start)/60, 'minutes', file=text_file)
text_file.close()

In [ ]:
%%time 

start = time.time()
path = '/home/60947081s/Air_temp_Experiment/'
floder_name = 'Air_ta_x160_y320_NumOfEns60_Timesteps150_NumOfComps3_SLIC_4096_100_7'
Slic_comp3 = TGMM(air_data, 60, 128, 150, max, min)
print('start get hist info')
Slic_comp3.get_hist_info()
hist_time = (time.time()-start)/60
print('SLIC')
Slic_comp3.SLIC(4096, 100, 7)
Slic_time = (time.time()-start)/60
print('fitting model')
Slic_comp3.fit_GMM(True, 3)
fitting_time = (time.time()-start)/60
Slic_comp3.save_data(path, floder_name)
time_txt = path+floder_name+'/'+'time.txt'

end = time.time()
text_file = open(time_txt,'w')
print('Hist times: ', hist_time, 'minutes', file=text_file)
print('SLIC times: ', Slic_time, 'minutes', file=text_file)
print('Fitting times: ', fitting_time, 'minutes', file=text_file)
print('All times: ', (end - start)/60, 'minutes', file=text_file)
text_file.close()

In [ ]:
%%time 

start = time.time()
path = '/home/60947081s/Air_temp_Experiment/'
floder_name = 'Air_ta_x160_y320_NumOfEns60_Timesteps150_NumOfComps3_SLIC_4096_100_7'
Slic_comp5 = TGMM(air_data, 60, 128, 150, max, min)
print('start get hist info')
Slic_comp5.get_hist_info()
hist_time = (time.time()-start)/60
print('SLIC')
Slic_comp5.SLIC(4096, 100, 7)
Slic_time = (time.time()-start)/60
print('fitting model')
Slic_comp5.fit_GMM(True, 5)
fitting_time = (time.time()-start)/60
Slic_comp5.save_data(path, floder_name)
time_txt = path+floder_name+'/'+'time.txt'

end = time.time()
text_file = open(time_txt,'w')
print('Hist times: ', hist_time, 'minutes', file=text_file)
print('SLIC times: ', Slic_time, 'minutes', file=text_file)
print('Fitting times: ', fitting_time, 'minutes', file=text_file)
print('All times: ', (end - start)/60, 'minutes', file=text_file)
text_file.close()

In [ ]:
%%time 

start = time.time()
path = '/home/60947081s/Air_temp_Experiment/'
floder_name = 'Air_ta_x160_y320_NumOfEns60_Timesteps150_NumOfComps3_SLIC_4096_100_7'
Slic_comp7 = TGMM(air_data, 60, 128, 150, max, min)
print('start get hist info')
Slic_comp7.get_hist_info()
hist_time = (time.time()-start)/60
print('SLIC')
Slic_comp7.SLIC(4096, 100, 7)
Slic_time = (time.time()-start)/60
print('fitting model')
Slic_comp7.fit_GMM(True, 7)
fitting_time = (time.time()-start)/60
Slic_comp7.save_data(path, floder_name)
time_txt = path+floder_name+'/'+'time.txt'

end = time.time()
text_file = open(time_txt,'w')
print('Hist times: ', hist_time, 'minutes', file=text_file)
print('SLIC times: ', Slic_time, 'minutes', file=text_file)
print('Fitting times: ', fitting_time, 'minutes', file=text_file)
print('All times: ', (end - start)/60, 'minutes', file=text_file)
text_file.close()

In [ ]:
%%time 

start = time.time()
path = '/home/60947081s/Air_temp_Experiment/'
floder_name = 'Air_ta_x160_y320_NumOfEns60_Timesteps150_NumOfComps1_SLIC_4096_100_9'
Slic_comp1 = TGMM(air_data, 60, 128, 150, max, min)
print('start get hist info')
Slic_comp1.get_hist_info()
hist_time = (time.time()-start)/60
print('SLIC')
Slic_comp1.SLIC(4096, 100, 9)
Slic_time = (time.time()-start)/60
print('fitting model')
Slic_comp1.fit_GMM(True, 1)
fitting_time = (time.time()-start)/60
Slic_comp1.save_data(path, floder_name)
time_txt = path+floder_name+'/'+'time.txt'

end = time.time()
text_file = open(time_txt,'w')
print('Hist times: ', hist_time, 'minutes', file=text_file)
print('SLIC times: ', Slic_time, 'minutes', file=text_file)
print('Fitting times: ', fitting_time, 'minutes', file=text_file)
print('All times: ', (end - start)/60, 'minutes', file=text_file)
text_file.close()

In [ ]:
%%time 

start = time.time()
path = '/home/60947081s/Air_temp_Experiment/'
floder_name = 'Air_ta_x160_y320_NumOfEns60_Timesteps150_NumOfComps3_SLIC_4096_100_9'
Slic_comp3 = TGMM(air_data, 60, 128, 150, max, min)
print('start get hist info')
Slic_comp3.get_hist_info()
hist_time = (time.time()-start)/60
print('SLIC')
Slic_comp3.SLIC(4096, 100, 9)
Slic_time = (time.time()-start)/60
print('fitting model')
Slic_comp3.fit_GMM(True, 3)
fitting_time = (time.time()-start)/60
Slic_comp3.save_data(path, floder_name)
time_txt = path+floder_name+'/'+'time.txt'

end = time.time()
text_file = open(time_txt,'w')
print('Hist times: ', hist_time, 'minutes', file=text_file)
print('SLIC times: ', Slic_time, 'minutes', file=text_file)
print('Fitting times: ', fitting_time, 'minutes', file=text_file)
print('All times: ', (end - start)/60, 'minutes', file=text_file)
text_file.close()

In [ ]:
%%time 

start = time.time()
path = '/home/60947081s/Air_temp_Experiment/'
floder_name = 'Air_ta_x160_y320_NumOfEns60_Timesteps150_NumOfComps3_SLIC_4096_100_9'
Slic_comp5 = TGMM(air_data, 60, 128, 150, max, min)
print('start get hist info')
Slic_comp5.get_hist_info()
hist_time = (time.time()-start)/60
print('SLIC')
Slic_comp5.SLIC(4096, 100, 9)
Slic_time = (time.time()-start)/60
print('fitting model')
Slic_comp5.fit_GMM(True, 5)
fitting_time = (time.time()-start)/60
Slic_comp5.save_data(path, floder_name)
time_txt = path+floder_name+'/'+'time.txt'

end = time.time()
text_file = open(time_txt,'w')
print('Hist times: ', hist_time, 'minutes', file=text_file)
print('SLIC times: ', Slic_time, 'minutes', file=text_file)
print('Fitting times: ', fitting_time, 'minutes', file=text_file)
print('All times: ', (end - start)/60, 'minutes', file=text_file)
text_file.close()

In [ ]:
%%time 

start = time.time()
path = '/home/60947081s/Air_temp_Experiment/'
floder_name = 'Air_ta_x160_y320_NumOfEns60_Timesteps150_NumOfComps3_SLIC_4096_100_9'
Slic_comp7 = TGMM(air_data, 60, 128, 150, max, min)
print('start get hist info')
Slic_comp7.get_hist_info()
hist_time = (time.time()-start)/60
print('SLIC')
Slic_comp7.SLIC(4096, 100, 9)
Slic_time = (time.time()-start)/60
print('fitting model')
Slic_comp7.fit_GMM(True, 7)
fitting_time = (time.time()-start)/60
Slic_comp7.save_data(path, floder_name)
time_txt = path+floder_name+'/'+'time.txt'

end = time.time()
text_file = open(time_txt,'w')
print('Hist times: ', hist_time, 'minutes', file=text_file)
print('SLIC times: ', Slic_time, 'minutes', file=text_file)
print('Fitting times: ', fitting_time, 'minutes', file=text_file)
print('All times: ', (end - start)/60, 'minutes', file=text_file)
text_file.close()